In [266]:
import pandas as pd
import numpy as np
import re
import unidecode

In [267]:
df = pd.read_csv('/home/daria/cislang/moldova.csv', sep=';')

In [268]:
df

,ru_name,en_name
0,"Чеботару, Евгений Сергеевич",Eugeniu Cebotaru
1,Сергей Викторович Алексеев,Serghei Alexeev
2,Эндрю Райель,Andrew Rayel
3,"Киртоакэ, Дорин",Dorin Chirtoacă
4,Александр Иванович Епуряну,Alexandru Epureanu
...,...,...
2136,Елена Гиржул,Elena Girzhul
2137,К. А. Бертини,K. A. Bertini
2138,"Королюк, Всеволод Емельянович",Vsevolod Coroliuc
2139,"Цельман, Цви",Zvi Zelman


In [276]:
def rus_name_align(name):
    mapper = {
            1: lambda name: ' '.join(name).strip(','),
            2: lambda name: ' '.join(name[::-1]).strip(','),
            3: lambda name: ' '.join(name[:-1][::-1]).strip(','),
            4: lambda name: ' '.join(name[:-2][::-1]).strip(','),
            5: lambda name: ' '.join(name[:-3][::-1]).strip(','),
            6: lambda name: ' '.join(name[:-6][::-1]).strip(','),
            7: lambda name: ' '.join(name[:-7][::-1]).strip(','),
        }
    if ',' in name:
        name = name.split()
        name = mapper[len(name)](name)
    return name

In [277]:
# deletes patronymic if present in the middle of the name

def align(name):
    name = name.split()
    if len(name) == 3:
        name = name[0] + ' ' + name[2]
    else:
        name = ' '.join(name)
    return name

In [278]:
def explode(name):
     return name.explode(ignore_index=True)
    
def remove_diacr(name):
    return name.apply(unidecode.unidecode)

In [279]:
def filt1(df):
    return (df
    .apply({'ru_name': rus_name_align, 'en_name': lambda x: x})
    .apply({'ru_name': align, 'en_name': align})
    .dropna()
    .reset_index(drop=True)
           )
    # fixed delete_incons here


In [280]:
# deletes rows with unequal length

def delete_incons(df):
    for i in range(len(df)):
        if df.ru_name.str.split()[i].__len__() != df.en_name.str.split()[i].__len__():
            df = df.drop(i)
    return df
# rewrite the function so .apply works (takes name as an argument, not df)

In [281]:
def filt2(df):
    return (df
    .apply(lambda item: item.str.split(" "))
    .apply({'ru_name': explode, 'en_name': explode})
    .apply(lambda item: item.str.upper(), axis=1)
    .reindex(columns=['en_name', 'ru_name'])
    .apply({'en_name': remove_diacr, 'ru_name': lambda x: x})
    .drop_duplicates()
    .to_csv('~/cislang/moldovian_tst.csv', sep=';', index=False, header=None)
           )

In [282]:
df = filt1(df)
df = delete_incons(df)
df = filt2(df)